In [21]:
import os
import numpy as np

from keras_preprocessing.image import ImageDataGenerator, array_to_img
from keras.applications.resnet import ResNet50
from keras.layers import Dense,Flatten
from keras.activations import softmax,relu
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.optimizers import adam_v2
from keras.losses import categorical_crossentropy


## unpickle the loaded file

In [3]:
# def unpickle(file):
#     import pickle
#     with open(file, 'rb') as fo:
#         dict = pickle.load(fo, encoding='bytes')
#     return dict
#
# data = unpickle('../data/meta')
# train_data = unpickle('../data/train')
# test_data = unpickle('../data/test')
#
# train_data

In [4]:
# images = []
#
# for i in range(train_data[b'data'].shape[0]):
#     images.append(np.transpose(np.reshape(train_data[b'data'][i], (3, 32, 32)), (1, 2, 0)))
#
# images = np.array(images)

In [5]:
# for i in range(len(train_data[b'fine_labels'])):
#     class_ = f"../data/not-vehicle/class_{train_data[b'coarse_labels'][i]}"
#     if not os.path.isdir(class_):
#         os.mkdir(class_)
#
#     dst = os.path.join(f"../data/not-vehicle/class_{train_data[b'coarse_labels'][i]}",
#                        str(train_data[b'filenames'][i].decode("utf-8")))
#     image = array_to_img(images[i])
#     image.save(dst)

## Reshape the Images

In [6]:
train_data = '../data/train'
test_data = '../data/test'

genarator = ImageDataGenerator(rescale=1./255,
                               rotation_range=45,
                               horizontal_flip=True,
                               vertical_flip=True,
                               validation_split=0.2)

train_gen = genarator.flow_from_directory(train_data,
                                          target_size=(224,224),
                                          batch_size=128,
                                          subset='training')

val_gen = genarator.flow_from_directory(train_data,
                                        target_size=(224,224),
                                        batch_size=128,
                                        subset='validation')

test_gen = genarator.flow_from_directory(test_data,
                                        target_size=(64,64),
                                        batch_size=128)

Found 40000 images belonging to 2 classes.
Found 9999 images belonging to 2 classes.
Found 420 images belonging to 2 classes.


## Using Resnet 50

In [8]:
base_model = ResNet50(weights='imagenet', input_shape=(224,224,3),include_top=False)

base_model.summary()

94781440/94765736 [==============================] - 552s 6us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                        

### Freezing the Layers

In [11]:
for layer in base_model.layers:
    layer.trainable = False

base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

### add clasifiers

In [15]:
last_layer = base_model.get_layer('conv5_block3_out')
last_output = last_layer.output

x = Flatten() (last_output)
x = Dense(units=512,activation=relu) (x)
Output_layer = Dense(units=2,activation=softmax,name='softmax')(x)

new_model = Model(inputs=base_model.input, outputs=Output_layer)

new_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [20]:
plot_model(new_model,
           show_shapes=True,
           show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [ ]:
new_model.compile(optimizer=adam_v2(),
              loss=categorical_crossentropy,
              metrics=['accuracy'])